In [19]:
import json
from functools import cmp_to_key
from inspect import getsource
from sys import argv
import re
import zlib
import base64

complete_match = ["""import json as a,sys
A,B,C=sys.argv
a.dump(dict(zip(*[sorted(U)for U in a.load(open(B))])),open(C,'w'))""",
                      """import json as a,sys,re
A,B,C=sys.argv
a.dump(dict(zip(*[sorted(U,key=lambda y:[int(c)if c.isdigit()else c for c in re.split('(\d+)',y)])for U in a.load(open(B))])),open(C,'w'))""",
                      """import json as a,sys,re
A,B,C=sys.argv
a.dump(dict(zip(*[sorted(U,key=lambda x:[[int(c)if c.isdigit()else c.replace('[','')for c in re.split('(\d+)',y)]for y in re.split('\/',x)])for U in a.load(open(B))])),open(C,'w'))""",
                  '']
    
incomplete_match = ["""import json as a,sys
X,Y,Z=sys.argv
A=[sorted(U)for U in a.load(open(Y))]
t={}
A[1]=[A[1][x+y]for x,y in enumerate([t[i]for i in range(0,len(t),2)for j in range(t[i+1])])] 
a.dump(dict(zip(*A)),open(Z,'w'))""",
                        '''import json as a,sys,re
X,Y,Z=sys.argv
A=[sorted(U,key=lambda y:[int(c)if c.isdigit()else c for c in re.split('(\d+)',y)])for U in a.load(open(Y))]
t={}
A[1]=[A[1][x+y]for x,y in enumerate([t[i]for i in range(0,len(t),2)for j in range(t[i+1])])] 
a.dump(dict(zip(*A)),open(Z,'w'))''',
                    '''import json as a,sys,re
X,Y,Z=sys.argv
A=[sorted(U,key=lambda x:[[int(c)if c.isdigit()else c.replace('[','')for c in re.split('(\d+)',y)]for y in re.split('\/',x)])for U in a.load(open(Y))]
t={}
A[1]=[A[1][x+y]for x,y in enumerate([t[i]for i in range(0,len(t),2)for j in range(t[i+1])])] 
a.dump(dict(zip(*A)),open(Z,'w'))''','']

order_key_scripts = ['''import json as a,sys,re
X,Y,Z=sys.argv
N=lambda x:[int(c)if c.isdigit()else c for c in re.split('(\d+)',x)]
def _(A):
    D=re.findall('{0}',A)
    if not D:
        return N(A)
    F=re.split('/',A) 
    O=[(i.find(D[0])!=-1)for i in F].index(True)
    T= '/'.join(F[:O-1])
    m=re.split('{1}',F[O-1])
    return N((T+'/'if T else'')+m[int(D[0][2:])-1])
a.dump(dict(zip(*[sorted(U,key=_)for U in a.load(open(Y))])),open(Z,'w'))''',
'''import json as a,sys,re
X,Y,Z=sys.argv
N=lambda x:[int(c)if c.isdigit()else c for c in re.split('(\d+)',x)]
def _(A):
    D=re.findall('{0}',A)
    if not D:
        return N(A)
    F=re.split('/',A) 
    O=[(i.find(D[0])!=-1)for i in F].index(True)
    T= '/'.join(F[:O-1])
    m=re.split('{1}',F[O-1])
    C=re.split('{2}',m[0])
    m[0]=m[0]if len(C)==1 else C[1]
    return N((T+'/'if T else'')+m[int(D[0][2:])-1])
a.dump(dict(zip(*[sorted(U,key=_)for U in a.load(open(Y))])),open(Z,'w'))''']

incomplete_order_key = ['''import json as a,sys,re
X,Y,Z=sys.argv
N=lambda x:[int(c)if c.isdigit()else c for c in re.split('(\d+)',x)]
def _(A):
    D=re.findall('{0}',A)
    if not D:
        return N(A)
    F=re.split('/',A) 
    O=[(i.find(D[0])!=-1)for i in F].index(True)
    T= '/'.join(F[:O-1])
    m=re.split('{1}',F[O-1])
    return N((T+'/'if T else'')+m[int(D[0][2:])-1])
t={2}
A=[sorted(U,key=_)for U in a.load(open(Y))]
A[1]=[A[1][x+y]for x,y in enumerate([t[i]for i in range(0,len(t),2)for j in range(t[i+1])])] 
a.dump(dict(zip(*A)),open(Z,'w'))''',
'''import json as a,sys,re
X,Y,Z=sys.argv
N=lambda x:[int(c)if c.isdigit()else c for c in re.split('(\d+)',x)]
def _(A):
    D=re.findall('{0}',A)
    if not D:
        return N(A)
    F=re.split('/',A) 
    O=[(i.find(D[0])!=-1)for i in F].index(True)
    T= '/'.join(F[:O-1])
    m=re.split('{1}',F[O-1])
    C=re.split('{2}',m[0])
    m[0]=m[0]if len(C)==1 else C[1]
    return N((T+'/'if T else'')+m[int(D[0][2:])-1])
t={3}
A=[sorted(U,key=_)for U in a.load(open(Y))]
A[1]=[A[1][x+y]for x,y in enumerate([t[i]for i in range(0,len(t),2)for j in range(t[i+1])])] 
a.dump(dict(zip(*A)),open(Z,'w'))''']

fake_scripts = ['''import json as a,sys,re
X,Y,Z=sys.argv
N=lambda x:[int(c)if c.isdigit()else c for c in re.split('(\d+)',x)]
def _(A):
    M=re.findall('{0}',A)
    if not M:
        return N(A)
    D=re.split('/',A) 
    O=[(i.find(M[0])!=-1)for i in D].index(True)
    temp='/'.join(D[:O-1])
    m=re.split('{1}',D[O-1])
    return N((temp+'/' if temp else '')+m[int(M[0][2:])-1]if A.find('{2}')==-1 and D[O-1].find('{1}')> -1 else A)
a.dump(dict(zip(*[sorted(U,key=_)for U in a.load(open(Y))])),open(Z,'w'))''',
'''import json as a,sys,re
X,Y,Z=sys.argv
N=lambda x:[int(c)if c.isdigit()else c for c in re.split('(\d+)',x)]
def _(A):
    M = re.findall('{0}',A)
    if not M:
        return N(A)
    D=re.split('/',A) 
    O=[(i.find(M[0])!=-1)for i in D].index(True)
    temp='/'.join(D[:O-1])
    m=re.split('{1}',D[O-1])
    C=re.split('{2}',m[0])
    m[0]=m[0]if len(C)==1 else C[1]
    return N((temp+'/' if temp else '')+m[int(M[0][2:])-1]if A.find('{3}')==-1 and D[O-1].find('{1}')> -1 else A)
a.dump(dict(zip(*[sorted(U,key=_)for U in a.load(open(Y))])),open(Z,'w'))''']

incomplete_fake = ['''import json as a,sys,re
X,Y,Z=sys.argv
N=lambda x:[int(c)if c.isdigit()else c for c in re.split('(\d+)',x)]
def _(A):
    M=re.findall('{0}',A)
    if not M:
        return N(A)
    D=re.split('/',A) 
    O=[(i.find(M[0])!=-1)for i in D].index(True)
    temp='/'.join(D[:O-1])
    m=re.split('{1}',D[O-1])
    return N((temp+'/' if temp else '')+m[int(M[0][2:])-1]if A.find('{2}')==-1 and D[O-1].find('{1}')> -1 else A)
t={3}
A=[sorted(U,key=_)for U in a.load(open(Y))]
A[1]=[A[1][x+y]for x,y in enumerate([t[i]for i in range(0,len(t),2)for j in range(t[i+1])])] 
a.dump(dict(zip(*A)),open(Z,'w'))''',
'''import json as a,sys,re
X,Y,Z=sys.argv
N=lambda x:[int(c)if c.isdigit()else c for c in re.split('(\d+)',x)]
def _(A):
    M = re.findall('{0}',A)
    if not M:
        return N(A)
    D=re.split('/',A) 
    O=[(i.find(M[0])!=-1)for i in D].index(True)
    temp='/'.join(D[:O-1])
    m=re.split('{1}',D[O-1])
    C=re.split('{2}',m[0])
    m[0]=m[0]if len(C)==1 else C[1]
    return N((temp+'/' if temp else '')+m[int(M[0][2:])-1]if A.find('{3}')==-1 and D[O-1].find('{1}')> -1 else A)
t={4}
A=[sorted(U,key=_)for U in a.load(open(Y))]
A[1]=[A[1][x+y]for x,y in enumerate([t[i]for i in range(0,len(t),2)for j in range(t[i+1])])] 
a.dump(dict(zip(*A)),open(Z,'w'))''']

map_in_name = ''
script_name = ''
order_signs = ['\$\w\d+','\w\$\d+']
order_sign = ''
separator = ''
prefix = ''
fake_separator = ''
need_swap = False
natural_key_flag = -1
zfill_flag = False
bit_size = -1
same_value_flag = False

def remove_white_space(ori):
    return "".join(str(ori).split(' '))

def shift_count(ori,new):
    count = 0
    for i in range(len(ori)):
        if ori[new[0][i]] != new[1][i]:
            count += 1
    return count

def generate_shift_table(ori,new):
    idx=[]
    if not new:
        return []
    
    index_dict = {}
    
    for i in range(len(ori)):
        target = ori[new[0][i]]
        if target != new[1][i]:
            index_dict.setdefault(new[1][i],i)
            if target in index_dict:
                idx.append(index_dict[target]-i)
            else:
                idx.append(new[1].index(target)-i)
        else:
            idx.append(0)
    return idx

def generate_run_length_table(idx):
    # [initialization]
    # add end flag
    idx += ['*']  
    # current run
    current = idx[0] 
    # current length
    run_length = 0
    # final output info
    res = []
    # readble output info
    readable = []
    
    # [run_length]
    for i in range(len(idx)):
        if idx[i] == current:
            run_length += 1
        else:
            res += [current,run_length]
            readable.append([current,run_length])
            current = idx[i]
            run_length = 1
    return res, readable

def two_level_run_length(table):
    flag = False
    temp = []
    tltable = []
    for i in range(0,len(table),2):
        if not flag and table[i+1] == 1:
            temp = [[table[i]],1]
            flag = True
        elif flag and table[i+1] == 1:
            temp[0] += [table[i]]
        else:
            flag = False
            tltable += temp
            tltable += table[i:i+2]
            temp = []
    tltable += temp
    return tltable

def swap_unsign_sign(oltable):
    return [oltable[i]*-1 if i % 2 == 0 else oltable[i] for i in range(len(oltable))]

def generate_swapsign_compression(origin_table,input_table):
    idx = generate_shift_table(origin_table,input_table)
    res, readable = generate_run_length_table(idx)
    cidx = base64.b85encode(zlib.compress(remove_white_space(idx).encode())).decode()
    cres = base64.b85encode(zlib.compress(remove_white_space(res).encode())).decode()
    cres = '''a.loads(zlib.decompress(base64.b85decode('%s')).decode())''' % cres.replace('\n','')
    rres = swap_unsign_sign(res)
    crres = base64.b85encode(zlib.compress(remove_white_space(rres).encode())).decode()
    crres = '''a.loads(zlib.decompress(base64.b85decode('%s')).decode())''' % crres.replace('\n','')
    return idx, res, readable, cidx, cres, rres, crres

def get_t_index(res, cres, rres, crres):
    targets = [remove_white_space(res),remove_white_space(rres),cres,crres]
    targets_ = [len(targets[i])if i < 2 else len(targets[i])+13 for i in range(len(targets))]
    t_index = targets_.index(min(targets_))
    return targets[t_index], t_index

#make choice between swapsign and compression, 
def make_choice(origin_table,input_table):
    idx, res, readable, cidx, cres, rres, crres = generate_swapsign_compression(origin_table,input_table)
    print('The most large consecutive segment length: %d.' % max(list(zip(*readable))[1]))
    print('Shift table plain text size: %d.' % len(remove_white_space(idx)))
    print('Shift table cipher text size: %d.' % len(cidx))
    print('One level run-length table plain text size: %d.' % len(remove_white_space(res)))
    print('One level run-length table cipher text size: %d.' % len(cres))
    print('After swapping, one level run-length table plain text size: %d.' % len(remove_white_space(rres)))
    print('After swapping, one level run-length table cipher text size: %d.' % len(crres))
    
    output_table, t_index = get_t_index(res, cres, rres, crres)
    targets_name = ['normal','swap-normal', 'compression','swap-compression']
    print('Choose %s.' % targets_name[t_index])
    # st, 1-rl, readable 1-rl, ot, tidx
    return idx, res, readable, output_table, t_index
    
def generate_tables(origin_table, input_table, method):
    global order_sign
    global prefix
    global separator
    global fake_separator
    global need_swap
    global natural_key_flag
    methods = ['Default', 'Natural', 'NatDir','Order']
    replacements = ["else c for","else c.lower()for","else c.upper()for","else c.swapcase()for",]
    choise, idx, res, readable, tltable, output_table = -1, [], [], [], [],''
    ns = shift_count(origin_table, input_table)
    if ns == 0:
        print('%s key can sovle this case.' % method)
        choise = methods.index(method)
        if method == methods[1]:
            complete_match[1] = complete_match[1].replace("else c for",replacements[natural_key_flag])
        if method == methods[3]:
            if not fake_separator:
                if prefix:
                    complete_match[3] = order_key_scripts[1].format(order_sign,separator,prefix)
                else:
                    complete_match[3] = order_key_scripts[0].format(order_sign,separator)
            else:
                if prefix:
                    complete_match[3] = fake_scripts[1].format(order_sign,separator,prefix,fake_separator)
                else:
                    complete_match[3] = fake_scripts[0].format(order_sign,separator,fake_separator)
        print('Script size using %s key: %d.' % (method,len(complete_match[methods.index(method)])))
    else:
        print('%s key can\'t sovle this case.' % method)
        print('Shift count in this method: %d.' % ns)
        idx, res, readable, output_table, t_index = make_choice(origin_table, input_table)
        need_swap = True if t_index % 2 else False
        if method == methods[1]:
            incomplete_match[1] = incomplete_match[1].replace("else c for",replacements[natural_key_flag])
        if method == methods[3]:
            if not fake_separator:
                if prefix:
                    incomplete_match[3] = incomplete_order_key[1].format(order_sign,separator,prefix,output_table)
                else:
                    incomplete_match[3] = incomplete_order_key[0].format(order_sign,separator,output_table)
            else:
                if prefix:
                    incomplete_match[3] = incomplete_fake[1].format(order_sign,separator,prefix,fake_separator,output_table)
                else:
                    incomplete_match[3] = incomplete_fake[0].format(order_sign,separator,fake_separator,output_table)
        else:
            incomplete_match[methods.index(method)]=incomplete_match[methods.index(method)].format(output_table)
        if t_index > 1:
            incomplete_match[methods.index(method)] = incomplete_match[methods.index(method)].replace('\n',',base64,zlib\n',1)
        if need_swap:
            incomplete_match[methods.index(method)] = incomplete_match[methods.index(method)].replace('x+y','x-y',1)
        print('Script size using %s key: %d.' % (method,len(incomplete_match[methods.index(method)])))
    return choise, idx, res, readable, tltable, output_table

def default_key_method(origin_mapping):
    keys, values = zip(*origin_mapping.items())
    keys, values = sorted(keys), sorted(values)
    return [keys,values]

def natural_key(x):
    return [int(c)if c.isdigit()else c for c in re.split('(\d+)',x)]

def natural_key_l(x):
    return [int(c)if c.isdigit()else c.lower() for c in re.split('(\d+)',x)]

def natural_key_u(x):
    return [int(c)if c.isdigit()else c.upper() for c in re.split('(\d+)',x)]

def natural_key_s(x):
    return [int(c)if c.isdigit()else c.swapcase() for c in re.split('(\d+)',x)]

def natural_key_method(origin_mapping):
    global natural_key_flag
    ori = [list(origin_mapping.keys()),list(origin_mapping.values())]
    funcs = [natural_key,natural_key_l,natural_key_u,natural_key_s] 
    nats = []
    scs = []     
    for i, func in enumerate(funcs):
        nats.append([sorted(x,key=func) for x in ori])
        scs.append( shift_count(origin_mapping, nats[-1]))
        if scs[-1] == 0:
            natural_key_flag = i
            break
    
    # format: 0.idx, 1.res, 2.readable, 3.cidx, 4.cres, 5.rres, 6.crres
    data = [[] for i in range(4) ]
    
    output_tables = []
    
    if natural_key_flag == -1:
        for i in range(4):
            data.append(generate_swapsign_compression(origin_mapping,nats[i]))
            output_tables.append(len(get_t_index(data[-1][1], data[-1][4], data[-1][5], data[-1][6])[0]))
        natural_key_flag = output_tables.index(min(output_tables))
    
    return nats[natural_key_flag]

def natdir_key(x):
    return [[int(c)if c.isdigit()else c.replace('[','')for c in re.split('(\d+)',y) ]for y in re.split('\/',x)]

def natdir_key_method(origin_mapping):
    ori = [list(origin_mapping.keys()),list(origin_mapping.values())]
    return [sorted(x,key=natdir_key) for x in ori]

def dimensions(string,i):
    it=list(re.finditer('\[([0-9]+)\]'*i,string))
    return (0,0) if not i else (i,it) if len(it) else dimensions(string,i-1)

def order_key(ori):
    K=ori[:]
    order=re.findall('\$[a-zA-Z](\d+)',ori)
    if order:
        d,ds = dimensions(ori,2)
        o=int(order[0])
        if d:
            K = ori[:ds[0].start()]+ds[-1 if len(ds) < o else o-1].group(0)+order[0]
        else:
            x_dim = list(re.finditer('[a-zA-Z]\d+\_\d+',ori))
            K= ori[:x_dim[0].start()]+x_dim[-1 if len(x_dim) < o else o-1].group(0)+order[0] if x_dim else ori
    else:
        K = ori
    return [int(c)if c.isdigit()else c for c in re.split('(\d+)',K)]

def vote(have_order, flag):
    global order_sign
    voting_result = dict()
    for k, v in have_order:
        Order=re.search(order_sign.replace('\\d+','(\\d+)'), k if flag else v)
        ks = re.split('/',k)
        vs = re.split('/',v)
        opos =[(i.find(Order.group(0))!=-1)for i in (ks if flag else vs)].index(True)
        folders = ks[opos-1] if flag else vs[opos-1] 
        folders = re.sub('\d+','',folders)
        folder = vs if flag else ks
        folder = re.sub('\d+','',folder[opos-1 if opos-1 < len(folder) else -1])
        patterns = re.split(re.escape(folder),folders)
        if patterns[0] == folders:
            continue
        for pattern in [i for i in patterns if i]:
            if pattern in voting_result:
                voting_result[pattern] += 1
            else:
                voting_result.setdefault(pattern,1)  
    return voting_result

def verify_prefix(have_order,flag,prefix):  
    print('Verify prefix: %s.' % prefix)
    global order_sign
    for k, v in have_order:
        Order=re.search(order_sign.replace('\\d+','(\\d+)'), k if flag else v)
        ks = re.split('/',k)
        vs = re.split('/',v)
        opos =[(i.find(Order.group(0))!=-1)for i in (ks if flag else vs)].index(True)
        folders = ks[opos-1] if flag else vs[opos-1] 
        folders = re.sub('\d+','',folders)
        folder = vs if flag else ks
        folder = re.sub('\d+','',folder[opos-1 if opos-1 < len(folder) else -1])
        patterns = re.split(re.escape(folder),folders)
        if patterns[0].find(prefix) == 0:
            return True
    return False

def verify_separator(have_order,flag,separator): 
    print('Verify separator: %s.' % separator)
    global order_sign
    fake_separators = {}
    for k, v in have_order:
        Order=re.search(order_sign.replace('\\d+','(\\d+)'), k if flag else v)
        ks = re.split('/',k)
        vs = re.split('/',v)
        opos =[(i.find(Order.group(0))!=-1)for i in (ks if flag else vs)].index(True)
        folders = ks[opos-1] if flag else vs[opos-1] 
        folders = re.sub('\d+','',folders)
        folder = vs if flag else ks
        folder = re.sub('\d+','',folder[opos-1 if opos-1 < len(folder) else -1])
        if folder == folders and folder.find(separator) > -1:
            patterns = re.split(separator,folder)
            fs = patterns[0]+separator
            if fs in fake_separators:
                fake_separators[fs] += 1
            else:
                fake_separators.setdefault(fs,1)
    return fake_separators

def find_separator_prefix(origin_mapping):
    global order_sign
    ko = [(k,v) for k, v in origin_mapping.items() if re.findall(order_sign,k)] 
    vo = [(k,v) for k, v in origin_mapping.items() if re.findall(order_sign,v)]
    if not (ko or vo):
        return '', '', ''
    flag = True
    print('Try to find order relation with A.')
    v1 = vote(ko, True)
    if v1:
        voting_result = v1
    else:
        print('Bad try.')
        print('Try to find order relation with B.')
        voting_result = vote(vo, False)
        flag = False
    if not voting_result:
        return '', '', ''
    print('Good try.')
    voting_result = sorted(list(voting_result.items()),key=lambda x:x[1],reverse=True)
    separator = voting_result[0][0]
    fake_separator = verify_separator((ko if flag else vo),flag,separator)
    fake_separator = sorted(list(fake_separator.items()),key=lambda x:x[1],reverse=True)
    fake_separator = fake_separator[0][0] if fake_separator else ''
    
    prefix = ''  
    if len(voting_result) > 1 and voting_result[0][0] not in voting_result[1][0]:
        prefix = voting_result[1][0]
    
    vvv = verify_prefix((ko if flag else vo),flag,prefix)
    if vvv:
        print("Have prefix: %s." % prefix)
    else:
        print("Found a bad prefix: %s." % prefix)
        prefix = ''
        
    return separator, prefix, fake_separator

def _order_key(ori,sep,pre):
    
    global order_sign
    new  =  ori[:]
    match = re.findall(order_sign,ori)
    if not match:
        new=ori
    else:
        m1=re.split('/',  ori) 
        opos =[(i.find(match[0])!=-1)for i in m1].index(True)
        temp = '/'.join(m1[:opos-1])
        m=re.split(sep,m1[opos-1])
        new=(temp+'/' if temp else '')+m[int(match[0][2:])-1]
    return [int(c)if c.isdigit()else c for c in re.split('(\d+)',new)]

def order_key_(ori,sep,pre): 
    global order_sign
    new  =  ori[:]
    match = re.findall(order_sign,ori)
    if not match:
        new=ori
    else:       
        m1=re.split('/',  ori) 
        opos =[(i.find(match[0])!=-1)for i in m1].index(True)
        temp = '/'.join(m1[:opos-1])
        m=re.split(sep,m1[opos-1])
        cut=re.split(pre,m[0])
        m[0] = m[0]if len(cut) == 1 else cut[1]
        new=(temp+'/' if temp else '')+m[int(match[0][2:])-1]
    return [int(c)if c.isdigit()else c for c in re.split('(\d+)',new)]

def _fake(ori,sep,pre,fake_separator):
    new  =  ori[:]
    match = re.findall(order_sign,ori)
    if not match:
        return [int(c)if c.isdigit()else c for c in re.split('(\d+)',ori)]
    m1=re.split('/',  ori) 
    opos =[(i.find(match[0])!=-1)for i in m1].index(True)
    temp = '/'.join(m1[:opos-1])
    m=re.split(sep,m1[opos-1])
    if ori.find(fake_separator) == -1 and m1[opos-1].find(sep) > -1:
        new=(temp+'/' if temp else '')+m[int(match[0][2:])-1]
    else:
        new = ori
    return [int(c)if c.isdigit()else c for c in re.split('(\d+)',new)]

def fake_(ori,sep,pre,fake_separator):
    new  =  ori[:]
    match = re.findall(order_sign,ori)
    if not match:
        return [int(c)if c.isdigit()else c for c in re.split('(\d+)',ori)]
    m1=re.split('/',  ori) 
    opos =[(i.find(match[0])!=-1)for i in m1].index(True)
    temp = '/'.join(m1[:opos-1])
    m=re.split(sep,m1[opos-1])
    m[0] = m[0][len(pre):] if m[0].find(pre) > -1 else m[0]
    if ori.find(fake_separator) == -1 and m1[opos-1].find(sep) > -1:
        new=(temp+'/' if temp else '')+m[int(match[0][2:])-1]
    else:
        new = ori
    return [int(c)if c.isdigit()else c for c in re.split('(\d+)',new)]

def order_key_method(origin_mapping):
    global order_sign
    global separator
    global prefix
    global fake_separator
    order_sign = order_signs[0]
    print('Using order sign: %s.' % order_sign)
    separator, prefix, fake_separator = find_separator_prefix(origin_mapping)
    print('Separator: %s. Prefix pattern: %s.' % ( (separator if separator else 'None'),(prefix if prefix else 'None') ) ) 
    if not(separator or prefix):
        print('Order sign \'%s\' has no use.' % order_sign)
        order_sign = order_signs[1]
        print('Using order sign: %s.' % order_sign)
        separator, prefix, fake_separator = '', '', ''
        separator, prefix, fake_separator = find_separator_prefix(origin_mapping)
        print('Separator: %s. Prefix pattern: %s.' % ( (separator if separator else 'None'),(prefix if prefix else 'None') ) ) 
    if not(separator or prefix):
        return []
    ori=[sorted(list(origin_mapping.keys())),sorted(list(origin_mapping.values()))]
    func=order_key_ if prefix else _order_key
    if fake_separator:
        func=fake_ if prefix else _fake
        print('There is a fake separate pattern: %s.' % fake_separator)
    else:
        print('Have no fake separate pattern.')
    _ori=[[ func(j,separator,prefix,fake_separator)if fake_separator else func(j,separator,prefix)for j in i] for i in ori]
    KV=[list(zip(ori[i],_ori[i])) for i in[0,1]]
    KV=[sorted(x,key=lambda x:x[1]) for x in KV]
    
    return [[z[0]for z in y]for y in KV]

def get_bit_size(origin_mapping):
    global zfill_flag
    max_size = -1
    for k, v in origin_mapping.items():
        values = re.findall('(\d+)',k)
        for value in values:
            if len(value) > 1 and value[0] == '0':
                zfill_flag = True
            else:
                bs = len(value)
                max_size = bs if bs > max_size else max_size
        values = re.findall('(\d+)',v)
        for value in values:
            if len(value) > 1 and value[0] == '0':
                zfill_flag = True
            else:
                bs = len(value)
                max_size = bs if bs > max_size else max_size  
    return max_size

def add_zfill_2_script(output_script_temp):
    if same_value_flag and output_script_temp.find('[int(c)') > -1:
        print('Need to pre-sort.')
        return output_script_temp.replace('\n','\ns=sorted\n',1).replace('[sorted(U','[s(s(U)')
    return output_script_temp

def same_value_checker(input_table,choice):
    nats = [natural_key,natural_key_l,natural_key_u,natural_key_s] 
    # sep, sep-pre, sep-fake, sep-pre-fake
    ords = [_order_key,order_key_,_fake,fake_]
    funcs = [lambda x:x,[],natdir_key,[]]
    #pure nat
    ordc = -1
    if choice == 1:
        funcs[choice] = nats[natural_key_flag]
    elif choice ==3:
        if not fake_separator:
            if not prefix:
                funcs[choice] = ords[0]
                ordc = 0
            else:
                funcs[choice] = ords[1]
                ordc = 1
        else:
            if not prefix:
                funcs[choice] = ords[2]
                ordc = 2
            else:
                funcs[choice] = ords[3]
                ordc = 3
    func = funcs[choice]
    for i in [0,1]:
        for j in range(len(input_table[0])-1):
            if ordc > -1:
                if ordc < 2:
                    if func(input_table[i][j],separator,prefix) == func(input_table[i][j+1],separator,prefix):
                        return True
                else:
                    if func(input_table[i][j],separator,prefix,fake_separator) == func(input_table[i][j+1],separator,prefix,fake_separator):
                        return True
            else:
                if func(input_table[i][j]) == func(input_table[i][j+1]):
                    return True
    return False

def check_type(origin_mapping,script_name):
    global complete_match
    global incomplete_match
    global order_key_scripts
    global bit_size
    global same_value_flag 
    bit_size = get_bit_size(origin_mapping)
    print("Bit size: %d." % bit_size)
    print("Zero leading issue occurred: %s." %('yes' if zfill_flag else 'no'))
    output_script = ""
    output_script_temp = ''
    choise = -1
    methods = ['Default', 'Natural', 'NatDir', 'Order']
    flags = ['Normal', 'lower', 'upper', 'swapcase']
    functions = [default_key_method, natural_key_method, natdir_key_method, order_key_method]
    data = [ [ [] for j in range(5) ] for i in range(len(methods)) ] # index, one level run-length, readable, two level run-length
    for i in range(len(methods)):
    #for i in range(1,2,1):
        if choise != -1:
            break
        else:
            input_table = functions[i](origin_mapping)   
            if not input_table:
                print('%s key can\'t apply to this case.' % methods[i])
                continue
            same_value_flag = same_value_checker(input_table,i)
            flag, data[i][0], data[i][1], data[i][2], data[i][3], data[i][4]  = generate_tables(origin_mapping, input_table, methods[i])
            choise = i if flag == i else -1
            script_tmp = complete_match[i] if flag == i else incomplete_match[i]
            script_tmp = add_zfill_2_script(script_tmp)
            if output_script_temp:
                if len(output_script_temp) > len(script_tmp):
                    output_script_temp = script_tmp
                    print('%s method is better.\nWrite out temporary script.' % methods[i]) 
                    #write_file(output_script_temp,script_name)
            else:
                output_script_temp = script_tmp
                print('%s method is better.\nWrite out temporary script.' % methods[i])
                #write_file(output_script_temp,script_name)
            if i == 1:
                print("Natural key flag: %s." % flags[natural_key_flag])
        print('***********************************************************')
    if choise != -1:
        output_script = complete_match[choise]
    else:
        print('No method can solve this case.')
        total= lambda x,y: len("".join(str(x).split(' ')))+len(y)
        two_level = [(methods[i],len(incomplete_match[i]))if data[i][0] else (methods[i],-1) for i in range(len(methods))] 
        two_level = list(filter(lambda x: x[1] > -1,sorted(two_level,key=lambda x:x[1])))
        t_index = methods.index(two_level[0][0])
        script_candidate = [ (len("".join(str(data[i][1]).split(' '))),len(data[i][4]),len(incomplete_match[i]))for i in range(len(methods)) ]
        print('Script candidate: %s.' % str(dict(zip(methods,script_candidate))) )
        print('Using %s key table to record the original relationship.' % methods[t_index])
        index, one_level, readable, two_level, output_table = data[t_index][0], data[t_index][1], data[t_index][2], data[t_index][3], data[t_index][4] 
        output_script = incomplete_match[t_index]
        if t_index == 1:
            print("Natural key flag: %s." % flags[natural_key_flag])
    print('Script size: %d.' % len(output_script))
    print('***********************************************************')
    output_script = add_zfill_2_script(output_script)
    return output_script
   
def write_file(output_script,script_name):
    script=open(script_name,'w')   
    script.write(output_script)
    script.close()
def test_all(case):
    print('case: {}'.format(case))
    map_in_name, script_name = '../cases/case{}.json'.format(case), 'test.py'
    map_in_file = open(map_in_name)
    origin_mapping = json.load(map_in_file)
    map_in_file.close()  
    #print('Script:\n%s\n'%check_type(origin_mapping))
    check_type(origin_mapping,script_name)
    print()
if __name__ == '__main__':
    ta = [0,1,2,3,4,5,6,7,8]
    tt=[1]
    for i in tt:
        test_all(i)


case: 1
Bit size: 4.
Zero leading issue occurred: no.
Default key can't sovle this case.
Shift count in this method: 10442.
The most large consecutive segment length: 429.
Shift table plain text size: 34646.
Shift table cipher text size: 3300.
One level run-length table plain text size: 20453.
One level run-length table cipher text size: 3324.
After swapping, one level run-length table plain text size: 20965.
After swapping, one level run-length table cipher text size: 3342.
Choose compression.
Script size using Default key: 3540.
Default method is better.
Write out temporary script.
***********************************************************
Natural key can't sovle this case.
Shift count in this method: 2956.
The most large consecutive segment length: 426.
Shift table plain text size: 28655.
Shift table cipher text size: 2603.
One level run-length table plain text size: 6969.
One level run-length table cipher text size: 2502.
After swapping, one level run-length table plain text size:

In [ ]:
test = """import json as a,sys,re
A,B,C=sys.argv
a.dump(dict(zip(*[sorted(U,key=lambda y:[int(c)if c.isdigit()else c for c in re.split('(\d+)',y)])for U in a.load(open(B))])),open(C,'w'))"""
test1 = """import json as a,sys,re
A,B,C=sys.argv
a.dump(dict(zip(*[sorted(U,key=lambda y:[int(c)if c.isdigit()else c for c in re.split('(\d+)',y)])for U in a.load(open(B))])),open(C,'w'))"""
print(len(test.replace('\n','\ns=sorted\n',1).replace('[sorted(U','[s(s(U)')))
print(len(test.replace('[sorted(U','[sorted(sorted(U)')))

In [ ]:
test1=['af',10]
test2=['af',10,10]
print(test1<test2)

In [18]:
test = ['123']
print(test.find('12'))

AttributeError: 'list' object has no attribute 'find'